In [1]:
from gbdxtools import Interface
from gbdxtools import CatalogImage
from gbdxtools.task import env
import json

gbdx = Interface()

In [2]:
catalog_id = env.inputs.get('catalog_id', '104001002884C200')
image = CatalogImage(catalog_id)
imgBounds = image.bounds
print(imgBounds)

(-77.74697382, 0.9785228165217394, -77.58496077652174, 2.04953586)


In [3]:
metadata = image.ipe.metadata['image']
#print(json.dumps(metadata, indent=4))

In [4]:
data_url = 's3://receiving-dgcs-tdgplatform-com/057792784010_01_003'

In [5]:
ao_task = gbdx.Task('AOP_Strip_Processor',
                    data=data_url,
                    bands='MS',
                    dra_mode='BaseLayerMatch')
ao_task.impersonation_allowed = True

In [ ]:
aoiBounds = [-77.740099, 0.980800, -77.582966, 1.291200]

print('image bounds', imgBounds)
print('chose bounds', aoiBounds)

bounds = [max(imgBounds[0], aoiBounds[0]), max(imgBounds[1], aoiBounds[1]),
                min(imgBounds[2], aoiBounds[2]), min(imgBounds[3], aoiBounds[3])]

print('fixed bounds', bounds)

In [ ]:
# problem: ship value may not be set
clip_task = gbdx.Task('gdalcrop',                      
                     #image='s3://gbd-customer-data/f9fbd84c-9463-4360-afeb-040cd9ea954b/lt_ao_bs_match1/',
                     image=ao_task.outputs.data.value,
                     crop='{} {} {} {}'.format(bounds[0], bounds[1], bounds[2], bounds[3]),
                     ship="false")

In [ ]:
tile_task = gbdx.Task('splitter-gbdx:0.0.5',
                     data_in=clip_task.outputs.cropped.value)

In [ ]:
fe_task = gbdx.Task('fe-gbdx:0.0.9',
                   data_in=tile_task.outputs.data_out.value)

In [ ]:
clustervis_task = gbdx.Task('clustervis-gbdx:0.0.13',
                           data_in=tile_task.outputs.data_out.value,
                           features_in=fe_task.outputs.data_out.value,
                           #data_in='s3://gbd-customer-data/f9fbd84c-9463-4360-afeb-040cd9ea954b/tiles2',
                           #features_in='s3://gbd-customer-data/f9fbd84c-9463-4360-afeb-040cd9ea954b/fe-out2',
                           train='true')

In [ ]:
#tasks = [ao_task]
#tasks2 = [clip_task]
#tasks = [ao_task, clip_task]
#tasks = [clip_task, tile_task]
#tasks = [fe_task]
#tasks = [clustervis_task]
#tasks = [clip_task, tile_task, fe_task, clustervis_task]
tasks = [ao_task, clip_task, tile_task, fe_task, clustervis_task]
workflow = gbdx.Workflow(tasks)

In [ ]:
workflow.savedata(ao_task.outputs.data, location='colombia-1-acomp')
workflow.savedata(clip_task.outputs.cropped, location='colombia-1-clip')
workflow.savedata(tile_task.outputs.data_out, location='colombia-1-tiles')
workflow.savedata(fe_task.outputs.data_out, location='colombia-1-fe')
workflow.savedata(clustervis_task.outputs.data_out, location='colombia-1-out2')

In [ ]:
workflow.execute()

In [ ]:
print workflow.id
print workflow.status
#print(json.dumps(workflow.events[len(workflow.events)-1], indent=4))
#print(json.dumps(workflow.events, indent=4))
for e in workflow.events:
    print("    {} {}".format(e['task'], e['event']))
    #print(e)
print(json.dumps(workflow.stdout, indent=4))
print(json.dumps(workflow.stderr, indent=4))

In [ ]:
workflow.cancel()